In [1]:
import numpy as np
import itertools

In [2]:
def diffsize(x, y):
    return np.sqrt(np.sum((x-y)**2))

vdiffsize = np.vectorize(diffsize, signature="(n),(n)->()")

In [3]:
x = np.random.rand(10, 3)
y = np.random.rand(10, 3)

In [4]:
for i in range(x.shape[0]):
    print(diffsize(x[i,:], y[i,:]))

0.7528007709107929
0.6081202249332686
0.9191688870519302
0.9139800957957566
0.3553731264452894
0.6130580069891284
0.8311462675137394
0.6735201803021905
0.5939851019640897
1.047918762185776


In [5]:
vdiffsize(x,y)

array([0.75280077, 0.60812022, 0.91916889, 0.9139801 , 0.35537313,
       0.61305801, 0.83114627, 0.67352018, 0.5939851 , 1.04791876])

# 行列化

`m[i,j] = f(x[i,:], y[j,:])` となるような行列を作る

## vectorize

In [6]:
def f(x, y):
    return x[0] + y[0]/10

dim = 2
numx = 3
numy = 4
vf = np.vectorize(f, signature="(n),(n)->()")
xs, ys = np.zeros((numx,dim)), np.zeros((numy,dim))
xs[:,0] = np.arange(numx)
ys[:,0] = np.arange(numy)
print(xs)
print(ys)

[[0. 0.]
 [1. 0.]
 [2. 0.]]
[[0. 0.]
 [1. 0.]
 [2. 0.]
 [3. 0.]]


In [7]:
# step1.
# ブロードキャストによって、必要な要素数になるまでコピーする.
# この時、最後の2つの次元が、xs,ysに合うように次元を調整する.
xs2, ys2 = np.zeros((numy,numx,dim)), np.zeros((numx,numy,dim))
xs2[:,:,:] = xs
ys2[:,:,:] = ys
# step2.
# xs2, ys2が同じ形になるように、転地によって次元を調整する.
xs2 = xs2.transpose((1,0,2))
vf(xs2, ys2)

array([[0. , 0.1, 0.2, 0.3],
       [1. , 1.1, 1.2, 1.3],
       [2. , 2.1, 2.2, 2.3]])

## broadcastのみ
vecorizeを使うと場合によっては激遅いので、ブロードキャストで頑張ってみる

In [8]:
xs3, ys3 = np.zeros((numy,numx,dim)), np.zeros((numx,numy,dim))
xs3[:,:,:] = xs
ys3[:,:,:] = ys
xs3 = xs3.transpose((1,0,2)).reshape(-1,dim)
ys3 = ys3.reshape(-1,dim)

In [9]:
mat = np.array([f(x,y) for x,y in zip(xs3, ys3)]).reshape(numx,numy)
mat

array([[0. , 0.1, 0.2, 0.3],
       [1. , 1.1, 1.2, 1.3],
       [2. , 2.1, 2.2, 2.3]])

## apply_along_axis化する

In [10]:
def argconcat(f, d):
    def ret(x):
        return f(x[:d], x[d:])
    return ret
f2 = argconcat(f, 2)

xys = np.concatenate([xs3, ys3], axis=1)
np.apply_along_axis(f2, 1, xys).reshape(numx, numy)

array([[0. , 0.1, 0.2, 0.3],
       [1. , 1.1, 1.2, 1.3],
       [2. , 2.1, 2.2, 2.3]])

## ベンチマークしてみる

In [11]:
def cross_prod0(xs, ys, f):
    N, M = len(xs), len(ys)
    ret = np.zeros((N, M))
    for i, j in itertools.product(range(N), range(M)):
        ret[i,j] = f(xs[i], ys[i])
    return ret

In [12]:
def cross_prod1(xs, ys, vf):
    dim = xs.shape[1]
    nx = xs.shape[0]
    ny = ys.shape[0]
    xs2, ys2 = np.zeros((ny, nx, dim)), np.zeros((nx, ny, dim))
    xs2[:,:,:] = xs
    ys2[:,:,:] = ys
    xs2 = xs2.transpose((1,0,2))
    return vf(xs2, ys2)

In [13]:
def cross_prod2(xs, ys, f):
    dim = xs.shape[1]
    nx = xs.shape[0]
    ny = ys.shape[0]
    xs2, ys2 = np.zeros((ny, nx, dim)), np.zeros((nx, ny, dim))
    xs2[:,:,:] = xs
    ys2[:,:,:] = ys
    xs2 = xs2.transpose((1,0,2)).reshape(-1,dim)
    ys2 = ys2.reshape(-1,dim)
    mat = np.array([f(x, y) for x, y in zip(xs2, ys2)]).reshape(nx, ny)
    return mat

In [14]:
def cross_prod3(xs, ys, f):
    dim = xs.shape[1]
    nx = xs.shape[0]
    ny = ys.shape[0]
    xs2, ys2 = np.zeros((ny, nx, dim)), np.zeros((nx, ny, dim))
    xs2[:,:,:] = xs
    ys2[:,:,:] = ys
    xs2 = xs2.transpose((1,0,2)).reshape(-1,dim)
    ys2 = ys2.reshape(-1,dim)
    xys = np.concatenate([xs2, ys2], axis=1)
    mat = np.apply_along_axis(f, 1, xys).reshape(nx, ny)
    return mat

In [15]:
dim = 2
nx, ny = 100, 100
xs = np.random.randn(nx, dim)
ys = np.random.randn(ny, dim)

In [16]:
%%timeit
cross_prod0(xs, ys, f)

9.9 ms ± 86.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
%%timeit
cross_prod1(xs, ys, vf)

22.5 ms ± 359 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
%%timeit
cross_prod2(xs, ys, f)

9.29 ms ± 150 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [19]:
%%timeit
cross_prod3(xs, ys, f2)

27.7 ms ± 164 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


クロージャーは遅いことが多い気がするので、グローバルの関数を定義してみる

In [20]:
def f3(x):
    return f(x[:2], x[2:])

In [21]:
%%timeit
cross_prod3(xs, ys, f3)

28 ms ± 725 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


あんま変わらん・・

## 結論
cross_prod2が一番早いが、シンプルなcross_prod0でもほとんど変わらない

### 結果が一致することを確認

In [22]:
print(cross_prod0(xs, ys, vf))
print(cross_prod1(xs, ys, vf))
print(cross_prod2(xs, ys, f))
print(cross_prod3(xs, ys, f2))
print(cross_prod3(xs, ys, f3))

[[-0.101323   -0.101323   -0.101323   ... -0.101323   -0.101323
  -0.101323  ]
 [-0.4045156  -0.4045156  -0.4045156  ... -0.4045156  -0.4045156
  -0.4045156 ]
 [ 0.94170384  0.94170384  0.94170384 ...  0.94170384  0.94170384
   0.94170384]
 ...
 [ 0.54846271  0.54846271  0.54846271 ...  0.54846271  0.54846271
   0.54846271]
 [-0.70333874 -0.70333874 -0.70333874 ... -0.70333874 -0.70333874
  -0.70333874]
 [ 1.45900788  1.45900788  1.45900788 ...  1.45900788  1.45900788
   1.45900788]]
[[-0.101323    0.01763912 -0.21205154 ... -0.09396191 -0.03468006
   0.00576916]
 [-0.52347772 -0.4045156  -0.63420626 ... -0.51611663 -0.45683477
  -0.41638556]
 [ 1.05243238  1.1713945   0.94170384 ...  1.05979347  1.11907532
   1.15952454]
 ...
 [ 0.54110163  0.66006374  0.43037309 ...  0.54846271  0.60774457
   0.64819379]
 [-0.76998168 -0.65101956 -0.88071022 ... -0.76262059 -0.70333874
  -0.66288952]
 [ 1.35191572  1.47087784  1.24118718 ...  1.35927681  1.41855867
   1.45900788]]
[[-0.101323    0.01